In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from Demos.win32cred_demo import target

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import numpy as np
import os
from tqdm import tqdm
import itertools
from itertools import combinations
import warnings
from numba import njit, prange
import time
from functools import wraps
warnings.filterwarnings("ignore")
def time_it(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()  # 记录开始时间
        result = func(*args, **kwargs)  # 调用被装饰的函数
        end_time = time.time()  # 记录结束时间
        execution_time = end_time - start_time  # 计算执行时间
        print(f"Function '{func.__name__}' executed in {execution_time:.4f} seconds")
        return result  # 返回函数结果
    return wrapper

In [2]:
import featuretools as ft
import pandas as pd
ORIGINAL_TRAIN = pd.read_csv('train.csv',engine = 'pyarrow')

In [3]:
ORIGINAL_TRAIN.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')

In [4]:
#提取目标字段
target=ORIGINAL_TRAIN['target']
ORIGINAL_TRAIN.drop(['target'],axis=1,inplace=True)

In [10]:
tram=ft.list_primitives()
tram
transform_factor=tram.query("type=='transform'")
aggregation_factor=tram.query("type=='aggregation'")
agg_primitives_all=aggregation_factor.iloc[:,0].values
trans_primitives_all=transform_factor.iloc[:,0].values
print(agg_primitives_all)

['kurtosis' 'time_since_last' 'max_consecutive_false' 'max_count'
 'time_since_last_min' 'max_consecutive_true' 'count_outside_range'
 'first_last_time_delta' 'count_less_than' 'n_most_common'
 'count_inside_nth_std' 'entropy' 'has_no_duplicates'
 'time_since_last_true' 'all' 'date_first_event' 'min' 'trend'
 'count_greater_than' 'is_monotonically_increasing' 'mean'
 'num_zero_crossings' 'time_since_first' 'median' 'variance'
 'count_above_mean' 'max' 'time_since_last_false' 'avg_time_between'
 'n_most_common_frequency' 'first' 'skew' 'mode' 'max_min_delta' 'std'
 'n_unique_days_of_calendar_year' 'max_consecutive_zeros' 'last'
 'num_unique' 'count_outside_nth_std' 'max_consecutive_positives'
 'num_true' 'num_false_since_last_true' 'count_inside_range'
 'is_monotonically_decreasing' 'n_unique_weeks'
 'num_consecutive_less_mean' 'average_count_per_unique' 'n_unique_days'
 'n_unique_days_of_month' 'num_consecutive_greater_mean' 'sum'
 'median_count' 'num_true_since_last_false' 'percent_un

In [16]:
df = df.fillna(0)

array([False])

In [ ]:
def featuretools_generate(df):
    # 构建实体
    es = ft.EntitySet(id = 'stocks')
    # 添加dataframe数据 
    es = es.add_dataframe(dataframe_name = 'stocks_data', dataframe = df, index = 'row_id')
    tram=ft.list_primitives()
    tram['type'].unique()
    agg_primitives=[
    'kurtosis', 'max_count', 
    'count_outside_range', 'n_most_common', 'count_inside_nth_std',
    'entropy', 'has_no_duplicates', 'min', 'trend', 'mean', 'median', 
    'variance', 'count_above_mean', 'max', 'first', 'skew', 'mode', 
    'max_min_delta', 'std', 'last', 'num_unique', 'count_outside_nth_std',
    'max_consecutive_positives', 'sum', 'median_count', 
    'percent_unique', 'min_count', 'num_peaks', 'count_below_mean', 'count',
    ]
    trans_primitives=[]
    print(es)
    feature_matrix, feature_names = ft.dfs(entityset=es, 
                                       target_dataframe_name = 'stocks_data', 
                                       max_depth = 1, 
                                       verbose = 1,
                                       agg_primitives=agg_primitives,
                                       trans_primitives=trans_primitives,
                                       n_jobs = 1)
    return feature_names,feature_matrix
feature_names,feature_matrix = featuretools_generate(ORIGINAL_TRAIN)

C:\Users\yigaa\.conda\envs\MLUsed\Lib\site-packages\featuretools\entityset\entityset.py:754: UserWarning: A Woodwork-initialized DataFrame was provided, so the following parameters were ignored: index
  warnings.warn(
C:\Users\yigaa\.conda\envs\MLUsed\Lib\site-packages\featuretools\synthesis\dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['count', 'count_above_mean', 'count_below_mean', 'count_inside_nth_std', 'count_outside_nth_std', 'count_outside_range', 'entropy', 'first', 'has_no_duplicates', 'kurtosis', 'last', 'max', 'max_consecutive_positives', 'max_count', 'max_min_delta', 'mean', 'median', 'median_count', 'min', 'min_count', 'mode', 'n_most_common', 'num_peaks', 'num_unique', 'percent_unique', 'skew', 'std', 'sum', 'trend', 'variance']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. 

Entityset: stocks
  DataFrames:
    stocks_data [Rows: 5237980, Columns: 16]
  Relationships:
    No relationships
Built 15 features
Elapsed: 00:01 | Progress:   0%|          

OPENFE
parallel
